In [ ]:
import os
import time
from glob import glob

import numpy as np
import xarray as xr
import rioxarray
import math

import satpy
from pyresample import load_area
from pyresample import geometry

import sys
sys.path.append("..")

from quadrilateral_interpolation import QuadInterpolationResampler

%matplotlib widget
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
   
def open_olci_file(dirname, bands=None):
    filenames = glob(os.path.join(dirname, "*.nc"))

    scene = satpy.Scene(reader="olci_l1b", filenames=filenames)

    variables = {
        #'solar_azimuth_angle': 'saa',
        # 'solar_zenith_angle': 'sza',
        # 'satellite_azimuth_angle': 'vaa',
        # 'satellite_zenith_angle': 'vza',
        # 'altitude': 'elevation',
        'longitude': 'longitude',
        'latitude': 'latitude'
    }        

    if bands is None:
        bands = range(1, 22)
    bands = [f'Oa{i:02}' for i in bands]
    scene.load(bands)

    scene.load(list(variables.keys()))
    scene.load([satpy.DataQuery(name=band, calibration='reflectance') for band in bands])

    dataset = {}
    for variable in variables:
        dataset[variable] =  scene[variable].compute()

    for band in bands:
        dataset[band] =  scene[band].compute()

    scene.unload()  # probably useless
    
    return xr.Dataset(dataset)

def swath_definition(dataset):
    source_def = geometry.SwathDefinition(lons=dataset['longitude'], lats=dataset['latitude'])
    return source_def


In [ ]:
# unzip an olci file in the olci-data directory
# download and unzip here the content of the archive S3A_OL_1_EFR____20170813T130925_20170813T131225_20180706T214618_0180_021_081_1980_LR2_R_NT_002.SEN3.zip or any other OLCI archive from Greenland.
# 
# It can be downloaded from: https://finder.creodias.eu/#
# 
# In the "path" input enter: "S3A_OL_1_EFR____20170813T130925_20170813T131225_20180706T214618_0180_021_081_1980_LR2_R_NT_002.SEN3"

filename = "olci-data/"

In [ ]:
my_area = load_area('greenland.yaml', 'greenland500')

my_area.pixel_upper_left

In [ ]:
t0 = time.time()

olci = open_olci_file(filename)
           
print("open file: ", time.time() - t0,"s")

In [ ]:
# for single band interpolation

source_def = swath_definition(olci)

t0 = time.time()

resampler = QuadInterpolationResampler(source_def, my_area)
regridded = resampler.resample(olci['Oa01'])

print("reprojection: ", time.time() - t0, "s")

In [ ]:
# for multiple bands with the same size/geometry, it is much faster to pass a single rank 3 xr.DataArray 
# rather than an xr.Dataset with several rank 2 images.

source_def = swath_definition(olci)

t0 = time.time()

dataarray = xr.concat([olci[f'Oa{i:02}'] for i in range(1, 21)], dim='band')

resampler = QuadInterpolationResampler(source_def, my_area)
regridded = resampler.resample(dataarray)

print("reprojection (rank 3 DataArray interpolation: ", time.time() - t0, "s")

t0 = time.time()

resampler = QuadInterpolationResampler(source_def, my_area)
regridded = resampler.resample(olci)

print("reprojection (multiple rank 2 interpolations): ", time.time() - t0, "s")

In [ ]:
def write_output(data, filename):
    # this functions write tif files based on a model file, here "Oa01"
    # opens a file for writing

    data.rio.to_raster(filename)
    
write_output(regridded['Oa01'], f"{filename}-quadinterp.tif")
print(regridded)

In [ ]:
plt.figure()
# plt.imshow(regridded.sel(band=20).squeeze())
plt.imshow(regridded['Oa01'].squeeze())